[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/05_mnist.ipynb)

# Vorbereitungen

Laufzeit-Typ ändern. Python-Pakete von fast.ai installieren und importieren.

In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.vision.all import *

# MNIST - die Mutter aller Datensätze :)

![MNIST](imgs/05_mnist.png)

Der MNIST Datensatz, ist vermutlich DER Datensatz, mit dem man die ersten Deep Learning Experimente macht. Er besteht aus handschriftlichen Ziffern von 0-9. Der Vorteil des Datensatzes ist, dass die Bilder schwarz / weiß, 28 x 28 Pixel groß und nur 10 Klassen (die Ziffern von 0 - 9) umfasst.

Anhand dieses Datensatzes können wir uns ansehen, wie wir mit PyTorch selbst Modell-Architekturen bauen können.

# Dataloader für MNIST

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
db = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock), 
                  get_items=get_image_files, 
                  get_y=parent_label)

dls = db.dataloaders(path)

In [ ]:
dls.show_batch()

# Das Modell

![LeNet5 Architecture](imgs/05_lenet-arch.png)

Wir benötigen folgende Layer für ein minimales CNN:

**Conv2d**
- in_channels = 1, Wieviele Kanäle (bei Bildern Farben) haben die Bilder? In unserem Fall s/w -> 1 Kanal
- out_channels = 6, Anzahl der Filter des Layers
- kernel_size = 3, wie groß sind die Filter des Layers

**ReLU**
Wir verwenden einen [ReLU](https://de.wikipedia.org/wiki/Rectifier_(neuronale_Netzwerke))-Layer als "non-linearity" 

**Average Pooling**
Ein `Pooling` Layer wird verwendet um die Größe innerhalb des Modells zu reduzieren. Ein Pooling-Layer mit der Kernel-Größe 2 halbiert den Output des vorherigen Layers.

- kernel_size=2

**Flatten**
Ein `Flatten` Layer nimmt den mehrdimensionalen Output des vorherigen Layers und reduziert ihn auf eine Dimension.

**Linear**
Ein `Linear` Layer führ eine Matrixmultiplikation (Output des vorherigen Layers * Parameter des Layers) durch.
- in_features = 1014, Größe des Outputs nach dem Flatten Layer, wie sich die 1014 ergeben sehen wir weiter unten
- out_features = 10, wir haben 10 Klassen deshalb muss der letzte Layer auch 10 Features ausgeben.

In [ ]:
l_conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)
l_relu = nn.ReLU()
l_avgpool = nn.AvgPool2d(kernel_size=2)

In [ ]:
batch = dls.one_batch()

In [ ]:
inp = batch[0].to('cpu')

In [ ]:
o = l_conv1(inp)

In [ ]:
o

In [ ]:
o = l_relu(o)

In [ ]:
o

In [ ]:
o.shape

In [ ]:
o = l_avgpool(o)

In [ ]:
o.shape

## Body

In [ ]:
body = nn.Sequential(l_conv1, l_relu, l_avgpool)

In [ ]:
o_body = body(inp)

In [ ]:
o_body.shape

## Head

Die Größe des Body-Outputs beträgt (6 x 13 x 13) nach dem Flatten Layer (3D -> 1D) beträgt die Größe also (6 x 13 x 13) => (1 x 1014)

In [ ]:
l_flat = nn.Flatten()
l_linear = nn.Linear(in_features=1014, out_features=10)

In [ ]:
head = nn.Sequential(l_flat,
                     l_linear)

In [ ]:
o_head = head(o_body)

In [ ]:
o_head.shape

## Model = Body + Head

In [ ]:
model = nn.Sequential(body, head)

In [ ]:
o = model(inp)

In [ ]:
o.shape

# Training

In [ ]:
learn = Learner(dls, model, metrics=accuracy)

In [ ]:
learn.summary()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-2)

In [ ]:
learn.show_results()